## Overall test data

This script read from the logs and predicts the overall performance of the model.

In [ ]:
import os
filename = "logs/2024-05-05_20-47-32_ms_7_bs_1_sl_4096.txt"
assert os.path.exists(filename), f"file {filename} does not exist"
from data_parser import parse
import re
timeinfo = parse(filename)
print(timeinfo)

ms = int(re.search(r"ms_(\d+)_", filename).group(1))
bs = int(re.search(r"bs_(\d+)_", filename).group(1))
sl = int(re.search(r"sl_(\d+).txt", filename).group(1))
print(ms, bs, sl)

In [ ]:
from model_info import ModelInfo
modelinfo = ModelInfo(ms)

In [ ]:
def estimate(timeinfo, modelinfo):
    # from platforms import PEAK
    PEAK = 312
    t = modelinfo.num_layers * (timeinfo["MLP"] + timeinfo["AttnLayer"] + timeinfo["RMS"] * 2 + timeinfo["Rotary"])
    mfu = sl * bs / t / PEAK * (6 * ms)
    for k in timeinfo:
        timeinfo[k] *= modelinfo.num_layers
    timeinfo["RMS"] *= 2
    return t, mfu, timeinfo

In [ ]:
t, mfu, timeinfo = estimate(timeinfo, modelinfo)
print(t, mfu, timeinfo)

In [ ]:
# plot the pie chart
import matplotlib.pyplot as plt
labels = [f"{k}: {v:.2f}ms" for k, v in timeinfo.items()]
sizes = [timeinfo["MLP"], timeinfo["AttnLayer"], timeinfo["RMS"] * 2, timeinfo["Rotary"]]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')